In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/ResNet")
#自定義資料取得
import ESC50_puliic_getDataset as epg
#自定義模組
import ESC50_puliic_model as epm

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#測試模型對於電鋸資料集的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    val_loss_array = []
    val_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #驗證準確度
        correct_val= 0
        #驗證資料總筆數
        total_val = 0
        
        for data in val_loader:
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #包裝參數 重組形狀
            val = Variable(inputs.view(input_shape))
            #向前傳播
            outputs = model(val)
            #計算loss
            val_loss = loss_func(outputs, labels)
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_val += len(labels)
            #計算判斷正確的筆數
            correct_val += (predicted == labels).float().sum()
        #計算這次驗證的準確度
        val_accuracy = 100 * correct_val / float(total_val)
        #紀錄這次驗證的準確度
        val_accuracy_array.append(val_accuracy)
        #紀錄這次驗證的loss
        val_loss_array.append(val_loss.data)
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, val_loss_array, val_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [3]:
train_loss_array = []
train_accuracy_array = []
val_loss_array = []
val_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]
#訓練秒數 1S-> 0 3S-> 1 5S-> 2
trainMode = 0
#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")
    #建立模型
    model = epm.ResNet_18(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    #loss方法
    loss_func = nn.CrossEntropyLoss()

    #資料抓入
    train, val = epg.getTrainDataset(fold, sound[trainMode], num_class)
    test_loader1 = epg.getTestDataset(sound[0])
    test_loader3 = epg.getTestDataset(sound[1])
    test_loader5 = epg.getTestDataset(sound[2])

    #分批方法
    train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
    val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)
    
    #訓練模型
    train_loss, train_accuracy, val_loss, val_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[trainMode], num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5)
    
    #將gpu資料取回
    train_loss = torch.stack(train_loss,dim=-1).cpu()
    train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
    val_loss = torch.stack(val_loss,dim=-1).cpu()
    val_accuracy = torch.stack(val_accuracy,dim=-1).cpu()
    
    for i in range(num_epochs):
        # 寫入資料
        f.write("fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold, i+1, num_epochs, train_loss[i], train_accuracy[i], val_loss[i], val_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    
    train_loss_array.append(train_loss)
    train_accuracy_array.append(train_accuracy)
    val_loss_array.append(val_loss)
    val_accuracy_array.append(val_accuracy)
    test1_accuracy_array.append(test1_accuracy)
    test3_accuracy_array.append(test3_accuracy)
    test5_accuracy_array.append(test5_accuracy)

f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.381544589996338 Train_acc: 43.500000% Val_Loss: 1.6078161001205444 Val_accuracy: 52.750000%
Test accuracy of the model: 70.200000%
Test accuracy of the model: 75.666667%
Test accuracy of the model: 81.000000%
Train Epoch: 2/30 Train_Loss: 1.2884069681167603 Train_acc: 57.562500% Val_Loss: 1.4952750205993652 Val_accuracy: 57.500000%
Test accuracy of the model: 75.400000%
Test accuracy of the model: 86.000000%
Test accuracy of the model: 92.000000%
Train Epoch: 3/30 Train_Loss: 1.1815314292907715 Train_acc: 63.875000% Val_Loss: 1.181370496749878 Val_accuracy: 61.250000%
Test accuracy of the model: 79.400000%
Test accuracy of the model: 90.000000%
Test accuracy of the model: 94.000000%
Train Epoch: 4/30 Train_Loss: 1.129409193

Train Epoch: 3/30 Train_Loss: 1.1259609460830688 Train_acc: 63.750000% Val_Loss: 1.422755479812622 Val_accuracy: 60.500000%
Test accuracy of the model: 78.600000%
Test accuracy of the model: 85.000000%
Test accuracy of the model: 91.000000%
Train Epoch: 4/30 Train_Loss: 1.0563862323760986 Train_acc: 69.562500% Val_Loss: 1.1123462915420532 Val_accuracy: 67.250000%
Test accuracy of the model: 87.800000%
Test accuracy of the model: 93.333333%
Test accuracy of the model: 98.000000%
Train Epoch: 5/30 Train_Loss: 0.8528929948806763 Train_acc: 71.875000% Val_Loss: 0.9601027965545654 Val_accuracy: 70.000000%
Test accuracy of the model: 93.800000%
Test accuracy of the model: 95.333333%
Test accuracy of the model: 98.000000%
Train Epoch: 6/30 Train_Loss: 0.8271878957748413 Train_acc: 73.812500% Val_Loss: 0.8242620229721069 Val_accuracy: 67.750000%
Test accuracy of the model: 89.800000%
Test accuracy of the model: 94.000000%
Test accuracy of the model: 98.000000%
Train Epoch: 7/30 Train_Loss: 0.8

Train Epoch: 6/30 Train_Loss: 0.7106658220291138 Train_acc: 75.937500% Val_Loss: 0.7678821086883545 Val_accuracy: 69.250000%
Test accuracy of the model: 96.200000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 7/30 Train_Loss: 0.6720602512359619 Train_acc: 76.875000% Val_Loss: 1.0421512126922607 Val_accuracy: 72.000000%
Test accuracy of the model: 95.800000%
Test accuracy of the model: 97.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 8/30 Train_Loss: 0.7450551390647888 Train_acc: 77.187500% Val_Loss: 0.930666446685791 Val_accuracy: 70.750000%
Test accuracy of the model: 97.600000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 9/30 Train_Loss: 0.5286667943000793 Train_acc: 80.687500% Val_Loss: 0.7289055585861206 Val_accuracy: 71.000000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 10/30 Train_Loss: 0

Train Epoch: 9/30 Train_Loss: 0.5174291729927063 Train_acc: 79.750000% Val_Loss: 1.1959494352340698 Val_accuracy: 77.500000%
Test accuracy of the model: 96.000000%
Test accuracy of the model: 98.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 10/30 Train_Loss: 0.4693922698497772 Train_acc: 82.000000% Val_Loss: 0.6641470789909363 Val_accuracy: 78.500000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 11/30 Train_Loss: 0.5928939580917358 Train_acc: 83.000000% Val_Loss: 0.7514573335647583 Val_accuracy: 80.500000%
Test accuracy of the model: 97.600000%
Test accuracy of the model: 98.666667%
Test accuracy of the model: 99.000000%
Train Epoch: 12/30 Train_Loss: 0.4965701997280121 Train_acc: 83.437500% Val_Loss: 0.8448554873466492 Val_accuracy: 80.000000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 13/30 Train_Loss

Train Epoch: 12/30 Train_Loss: 0.44483351707458496 Train_acc: 84.625000% Val_Loss: 0.5983625054359436 Val_accuracy: 72.000000%
Test accuracy of the model: 96.400000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 13/30 Train_Loss: 0.5528486371040344 Train_acc: 84.062500% Val_Loss: 1.1548315286636353 Val_accuracy: 73.500000%
Test accuracy of the model: 96.800000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 14/30 Train_Loss: 0.5751332640647888 Train_acc: 85.250000% Val_Loss: 1.1650525331497192 Val_accuracy: 72.500000%
Test accuracy of the model: 96.600000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 15/30 Train_Loss: 0.33183643221855164 Train_acc: 86.812500% Val_Loss: 0.7710798382759094 Val_accuracy: 72.500000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 16/30 Train

In [4]:
train_loss_last = []
train_accuracy_last = []
val_loss_last = []
val_accuracy_last = []
test1_accuracy_last = []
test3_accuracy_last = []
test5_accuracy_last = []

#設定存檔位置
f = open("lastData.txt", "w")

#整理出訓練完的數值
for fold in range(5):
    train_loss_last.append(train_loss_array[fold][num_epochs-1])
    train_accuracy_last.append(train_accuracy_array[fold][num_epochs-1])
    val_loss_last.append(val_loss_array[fold][num_epochs-1])
    val_accuracy_last.append(val_accuracy_array[fold][num_epochs-1])
    test1_accuracy_last.append(test1_accuracy_array[fold][num_epochs-1])
    test3_accuracy_last.append(test3_accuracy_array[fold][num_epochs-1])
    test5_accuracy_last.append(test5_accuracy_array[fold][num_epochs-1])
    # 寫入資料
    f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][num_epochs-1], train_accuracy_array[fold][num_epochs-1], val_loss_array[fold][num_epochs-1], val_accuracy_array[fold][num_epochs-1], test1_accuracy_array[fold][num_epochs-1], test3_accuracy_array[fold][num_epochs-1], test5_accuracy_array[fold][num_epochs-1]))
f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(val_loss_last), max(val_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
f.write("Min Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(val_loss_last), min(val_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(val_loss_last), np.mean(val_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
f.close()